In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time

import sys
sys.path.append('../../')

from data.processing import get_data

import models

from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new, predict_r0_multipliers, save_r0_mul
from main.seir.sensitivity import calculate_sensitivity_and_plot
from utils.generic.create_report import save_dict_and_create_report
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
#from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles, plot_r0_multipliers

import yaml

In [2]:
place_list = [('Jharkhand', None), ('Jharkhand', 'Bokaro'), ('Jharkhand', 'Dhanbad'), ('Jharkhand', 'East Singhbhum')]
pop_list = [3.772e+7, 2.36e+6, 3.07e+6, 2.62e+6]

In [3]:
config_list = ['jharkhand.yaml', 'bokaro.yaml', 'dhanbad.yaml', 'east singhbhum.yaml']

In [4]:
config_filename = 'dhanbad.yaml'
config = read_config(config_filename)

In [6]:
for i, place in enumerate(place_list):

    config_filename = config_list[i]
    config = read_config(config_filename)

    '''
    # READING THE DEFAULT CONFIG FILE
    config_filename = 'dhanbad.yaml'
    config = read_config(config_filename)
    
    # UPDATING THE TARGET LOCATION
    state, district = place
    config['fitting']['data']['dataloading_params']['state'] = state
    config['fitting']['data']['dataloading_params']['district'] = district
    config['fitting']['default_params']['N'] = pop_list[i]
    '''

    state, district = place
    config['fitting']['data']['dataloading_params']['state'] = state
    config['fitting']['data']['dataloading_params']['district'] = district
    config['fitting']['default_params']['N'] = pop_list[i]

    print("RUNNING ONCALL {} FOR {} - {}".format(i + 1, state, district))

    predictions_dict = {}

    output_folder = '../../misc/reports/{}_{}-{}'.format(datetime.datetime.now().strftime("%Y_%m%d_%H%M%S"), state, district)

    train_period, val_period, test_period = config['fitting']['split']['train_period'], config['fitting']['split']['val_period'], config['fitting']['split']['test_period']

    m0_params = copy.deepcopy(config['fitting'])
    m1_params = copy.deepcopy(config['fitting'])
    m2_params = copy.deepcopy(config['fitting'])

    m0_params['split']['val_period'] = test_period
    m0_params['split']['test_period'] = 0

    m1_params['split']['val_period'] = 0
    m1_params['split']['test_period'] = 0

    m2_params['split']['test_period'] = 0

    predictions_dict['m0'] = single_fitting_cycle(**m0_params)
    predictions_dict['m1'] = single_fitting_cycle(**m1_params) 
    predictions_dict['m2'] = single_fitting_cycle(**m2_params)

    predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")

    predictions_dict['m0']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, config, which_fit='m0')
    predictions_dict['m1']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, config, which_fit='m1')
    predictions_dict['m2']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, config, which_fit='m2')
    
    # M0 based forecasting for the next train_period + val_period
    predictions_dict['m0']['forecasts'] = {}
    predictions_dict['m0']['forecasts']['best'] = get_forecast(predictions_dict, train_fit='m0', 
                                                               model=config['fitting']['model'], 
                                                               days=config['forecast']['forecast_days'])
    
    # M1 based forecasting for the next train_period + forecast_period (no validation period)
    predictions_dict['m1']['forecasts'] = {}
    predictions_dict['m1']['forecasts']['best'] = get_forecast(predictions_dict, train_fit='m1', 
                                                               model=config['fitting']['model'], 
                                                               days=config['forecast']['forecast_days'])


    # M2 based forecasting for the next train_period + val_period + forecast_period (no validation period)
    predictions_dict['m2']['forecasts'] = {}
    predictions_dict['m2']['forecasts']['best'] = get_forecast(predictions_dict, train_fit='m1', 
                                                               model=config['fitting']['model'], 
                                                               days=config['forecast']['forecast_days'])


    # Predictions plotted are for the most recent train_period days + test_period + forecast_period (no validation_period)
    predictions_dict['m1']['plots']['forecast_best'] = plot_forecast(predictions_dict,
                (config['fitting']['data']['dataloading_params']['state'], config['fitting']['data']['dataloading_params']['district']),
                error_bars=True)

    # Predictions plotted are for the most recent train_period days + test_period + forecast_period (no validation_period)
    predictions_dict['m2']['plots']['forecast_best'] = plot_forecast(predictions_dict,
                (config['fitting']['data']['dataloading_params']['state'], config['fitting']['data']['dataloading_params']['district']), which_fit = 'm2',
                error_bars=True)


    predictions_dict['m1']['trials_processed'] = forecast_all_trials(predictions_dict, train_fit='m1', 
                                                                 model=config['fitting']['model'], 
                                                                 forecast_days=config['forecast']['forecast_days'])


    predictions_dict['m2']['trials_processed'] = forecast_all_trials(predictions_dict, train_fit='m2', 
                                                                 model=config['fitting']['model'], 
                                                                 forecast_days=config['forecast']['forecast_days'])

    kforecasts = plot_top_k_trials(predictions_dict, train_fit='m1',
                               k=config['forecast']['num_trials_to_plot'],
                               which_compartments=config['forecast']['plot_topk_trials_for_columns'])


    predictions_dict['m1']['plots']['forecasts_topk'] = {}
    for column in config['forecast']['plot_topk_trials_for_columns']:
        predictions_dict['m1']['plots']['forecasts_topk'][column.name] = kforecasts[column]


    uncertainty_args = {'predictions_dict': predictions_dict, **config['uncertainty']['uncertainty_params']}
    uncertainty = config['uncertainty']['method'](**uncertainty_args)

    uncertainty_forecasts = uncertainty.get_forecasts()
    for key in uncertainty_forecasts.keys():
        predictions_dict['m2']['forecasts'][key] = uncertainty_forecasts[key]['df_prediction']
    
    predictions_dict['m2']['forecasts']['ensemble_mean'] = uncertainty.ensemble_mean_forecast

    predictions_dict['m2']['beta'] = uncertainty.beta
    predictions_dict['m2']['beta_loss'] = uncertainty.beta_loss
    predictions_dict['m2']['deciles'] = uncertainty_forecasts
    
    predictions_dict['m2']['plots']['forecast_best_50'] = plot_forecast(predictions_dict, 
                                                                        (config['fitting']['data']['dataloading_params']['state'], 
                                                                         config['fitting']['data']['dataloading_params']['district']),
                                                                         fits_to_plot=['best', 50], error_bars=False)
    predictions_dict['m2']['plots']['forecast_best_80'] = plot_forecast(predictions_dict, 
                                                                        (config['fitting']['data']['dataloading_params']['state'], 
                                                                         config['fitting']['data']['dataloading_params']['district']),
                                                                         fits_to_plot=['best', 80], error_bars=False)
    predictions_dict['m2']['plots']['forecast_ensemble_mean_50'] = plot_forecast(predictions_dict, 
                                                                                 (config['fitting']['data']['dataloading_params']['state'], 
                                                                                  config['fitting']['data']['dataloading_params']['district']),
                                                                                  fits_to_plot=['ensemble_mean', 50], error_bars=False)

    ptiles_plots = plot_ptiles(predictions_dict, which_compartments=config['forecast']['plot_ptiles_for_columns'])
    predictions_dict['m2']['plots']['forecasts_ptiles'] = {}
    for column in config['forecast']['plot_ptiles_for_columns']:
        predictions_dict['m2']['plots']['forecasts_ptiles'][column.name] = ptiles_plots[column]
    
    
    r0_mul_params= predictions_dict['m2']['deciles'][config['what_ifs']['percentile']]['params']
    
    predictions_dict['m2']['r0_mul_dict'] = predict_r0_multipliers(predictions_dict, r0_mul_params, model=config['fitting']['model'], days=config['forecast']['forecast_days'], multipliers=config['what_ifs']['multipliers'])
    
    predictions_dict['m2']['plots']['r0_mul_dict'] = plot_r0_multipliers(predictions_dict, None, predictions_dict['m2']['r0_mul_dict'], None, config['fitting']['data']['dataloading_params']['state'], config['fitting']['data']['dataloading_params']['district'])

    save_dict_and_create_report(predictions_dict, config, ROOT_DIR=output_folder, config_filename=config_filename)

    df_output = create_decile_csv_new(predictions_dict)
    df_output.to_csv(f'{output_folder}/deciles.csv')

    df_all = create_all_trials_csv(predictions_dict)
    df_all.to_csv(f'{output_folder}/all_trials.csv')

    save_r0_mul(predictions_dict['m2']['r0_mul_dict'], output_folder)

    predictions_dict['m1']['df_district'].to_csv(f'{output_folder}/df_district.csv')

    print('DONE WITH ONCALL {}'.format(i + 1)) 
    break

RUNNING ONCALL 1 FOR Jharkhand - None
Performing m1 fit ..
loading from ../../misc/cache/dataframes_ts_20112020_tracker.pkl
splitting data ..
splitting data ..
          date      state   total  active  recovered  deceased     tested  \
0   2020-04-26  Jharkhand      82      66         13         3     7806.0   
1   2020-04-27  Jharkhand     103      83         17         3     8757.0   
2   2020-04-28  Jharkhand     105      83         19         3     9408.0   
3   2020-04-29  Jharkhand     107      85         19         3    10268.0   
4   2020-04-30  Jharkhand     110      88         19         3    10987.0   
..         ...        ...     ...     ...        ...       ...        ...   
204 2020-11-16  Jharkhand  106230    2754     102548       928  3807997.0   
205 2020-11-17  Jharkhand  106491    2669     102891       931  3826168.0   
206 2020-11-18  Jharkhand  106742    2637     103171       934  3846092.0   
207 2020-11-19  Jharkhand  106972    2600     103435       937  386646


getting forecasts ..
getting forecasts ..
getting forecasts ..
15it [00:00, 143.19it/s]getting forecasts ..
1000it [00:08, 117.17it/s]
13it [00:00, 128.46it/s]getting forecasts ..
1000it [00:07, 126.44it/s]
100%|██████████| 100/100 [00:48<00:00,  2.04trial/s, best loss: 1.7741169887836816]
getting forecasts ..
getting forecasts ..
getting forecasts ..
getting forecasts ..
getting forecasts ..
getting forecasts ..


KeyError: 'forecast_ensemble_mean_best'